# 🐍➡️ Go: Python to Go Code Converter

This notebook demonstrates an AI-powered tool for converting Python function signatures to Go code using multiple advanced models via OpenRouter.

**Key Capabilities:**
- 🔄 Converts Python function signatures to Go equivalents using AI models
- 🤖 Integrates multiple AI models (OpenAI, Gemini, Grok) for code conversion
- 🖥️ Provides a user-friendly Gradio web interface for interactive use

Explore the cells below to see how Python code is parsed, mapped to Go, and how conversion results are generated and scored using different models.

### Step 1: Import Required Libraries
This cell imports the necessary libraries for the Python to Go code generator.

In [ ]:
# imports
import os
import re
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

### Step 2: Load OpenRouter API key 
This cell imports Open router key from .env file

In [ ]:
load_dotenv()

# Check for API key
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise EnvironmentError("OPENROUTER_API_KEY not found in .env file")

print("✅ Environment loaded successfully")

### Step 3: Iinitailizing Open Ruter client
This cell initializes Open Router Client

In [ ]:
# Initialize OpenRouter client
openrouter = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

print("✅ OpenRouter client initialized")

### Step 4: Prompt designing
This cell prepares appropriate system prompts 

In [ ]:
SYSTEM_PROMPT = """
Convert the provided Python code into efficient and idiomatic Go.

Requirements:
- Return ONLY Go code
- Must compile if pasted into a Go compiler
- Include package main and func main()
- Preserve logic exactly
- Use Go standard library only
- Use idiomatic Go style
"""

### Step 5: Models Listing

In [ ]:
MODELS = {
    "OpenAI (GPT-4o Mini)": "openai/gpt-4o-mini",
    "Gemini Flash": "google/gemini-2.5-flash",
    "Grok Beta": "x-ai/grok-3-mini",
    "Claude (Anthropic)": "anthropic/claude-opus-4.6",
    "DeepSeek V3": "deepseek/deepseek-r1"
}

print(f"✅ Loaded {len(MODELS)} models")

### Step 6: Function that converts python code to Go Lang

In [ ]:
def convert_to_go(model_id, python_code):
    try:
        response = openrouter.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": python_code}
            ],
            temperature=0
        )
        return response.choices[0].message.content.strip(), None
    except Exception as e:
        return None, str(e)

### Step 7: This cell visualizes the results 

It benchmark the results and compares conversion accuracy.

In [ ]:
def score_go_code(go_code, python_code):
    score = 0
    notes = []

    if "package main" in go_code:
        score += 2
        notes.append("✔ package main present")
    else:
        notes.append("✘ missing package main")

    if "func main()" in go_code:
        score += 2
        notes.append("✔ main function present")
    else:
        notes.append("✘ missing main function")

    if "fmt." in go_code:
        score += 1
        notes.append("✔ printing preserved")

    math_ops = ["*", "/", "+", "-"]
    for op in math_ops:
        if op in python_code and op in go_code:
            score += 1
            notes.append(f"✔ operator '{op}' preserved")

    py_vars = set(re.findall(r'\b[a-zA-Z_]\w*\b', python_code))
    go_vars = set(re.findall(r'\b[a-zA-Z_]\w*\b', go_code))
    shared = py_vars.intersection(go_vars)

    if shared:
        score += len(shared) * 0.3
        notes.append("✔ variables preserved: " + ", ".join(list(shared)[:5]))

    if "TODO" in go_code or "..." in go_code:
        score -= 3
        notes.append("✘ incomplete code detected")

    if "print(" in go_code:
        score -= 2
        notes.append("✘ Python print found")

    return score, notes

## Step 8: This function provides a user interface for interactive Python-to-Go code conversion using Gradio.

In [ ]:
def evaluate_models(python_code, selected_models):
    results = []

    for name in selected_models:
        model_id = MODELS[name]
        go_code, error = convert_to_go(model_id, python_code)

        if error:
            results.append({
                "model": name,
                "error": error
            })
            continue

        score, notes = score_go_code(go_code, python_code)

        results.append({
            "model": name,
            "go_code": go_code,
            "score": score,
            "notes": notes,
            "error": None
        })

    valid = [r for r in results if r["error"] is None]
    winner = max(valid, key=lambda x: x["score"]) if valid else None

    return results, winner

## Step 9: This function process the above functions as a gradio tool 

In [ ]:
def process_code(python_code, selected_models):
    results, winner = evaluate_models(python_code, selected_models)

    output = ""

    for r in results:
        output += f"\nMODEL: {r['model']}\n"
        output += "-"*40 + "\n"

        if r["error"]:
            output += f"❌ Error:\n{r['error']}\n\n"
        else:
            output += r["go_code"] + "\n\n"
            output += f"Score: {r['score']:.2f}\n"
            output += "\n".join(r["notes"]) + "\n\n"

    if winner:
        output += f"\nBest model: {winner['model']} (Score: {winner['score']:.2f})\n"

    return output

## Step 10: Gradio Code that displays the implementation

In [ ]:
interface = gr.Interface(
    fn=process_code,
    inputs=[
        gr.Textbox(lines=12, label="Paste Python Code"),
        gr.CheckboxGroup(
            choices=list(MODELS.keys()),
            value=[list(MODELS.keys())[0]],
            label="Select Models"
        )
    ],
    outputs=gr.Textbox(label="Go Conversion Results"),
    title="Python → Go Generator (Smart Evaluation)",
    description="Compare AI model translations and select the best Go code."
)
interface.launch()